<center> <h1>Creating essential functions and classes</h1> </center>

---

In [2]:
from cmath import  exp,  phase
from contextlib import redirect_stderr
import numpy as np
from math import pi, log, sqrt, sin , cos, tan, asin, acos, atan, atanh
from scipy.integrate import quad, dblquad
from sympy.physics.wigner import wigner_9j, wigner_3j, clebsch_gordan
from sympy import Symbol , S
from decimal import *
# from scipy import integrate
# from decimal import Decimal
from scipy.special import sph_harm 
from math import factorial as fact
# from mom_mesh import trns as mom_grid
import matplotlib.pyplot as plt
import time
import pandas as pd
"""Numerical integration mesh routines
"""
from typing import Tuple
from numpy.polynomial.legendre import leggauss
from random import random

from itertools import islice
from os.path import exists
from copy import deepcopy
%matplotlib inline

## constants

In [3]:
# chiPT constants (MeV)

m_proton    = 938.2720
m_neutron   = 939.5653
mpi         = 139.5702
mu          = (m_proton * m_neutron)/(m_proton + m_neutron)
m_n         =  2 * (m_proton * m_neutron)/(m_proton + m_neutron)

gA  = 1.27
fpi = 92.4
g0,epsilon  = 1.0 , 10**-4

# unit conversion constants
hc = 197.327052

# to create angle and momentum grids
n_angle,n1, n2, p1, p2, p3, q0 = 15, 80, 80, 150,650, 10300, 100


### wigner 9-j symbols


In [4]:
class W9J:
    _cache = {}

    def __call__(self, *args):
        res = self._cache.get(args, None)  # return None if args keyword does not exist
        if res is None:
            res =  float(wigner_9j(*[S(i)  for i in args]))
            self._cache[args] = res
        
        return res

'''
w9j(
     j1, j2, j3,
     j4, j5, j6,
     j7, j8, j9
)
'''
w9j = W9J()

In [5]:
# # Example

# w9j(1,1,2,1,1,0,2,2,2)

### wigner 3-j symbols


In [6]:
class W3J:
    _cache = {}

    def __call__(self, *args):
        res = self._cache.get(args, None)  # return None if args keyword does not exist
        if res is None:
            res = float(wigner_3j(*[S(i)  for i in args]))
            self._cache[args] = res
        
        return res

'''
w3j(
     j1, j2, j3,
     m1, m2, m3
)
'''
w3j = W3J()

In [7]:
# # Example

# w3j(1/2,1/2,1,1/2,1/2,-1)

### Clebsh-gorden coefficients

In [8]:
class ClebschGordan:
    _cache = {}

    def __call__(self, *args):
        res = self._cache.get(args, None)
        if res is None:
            res = float(clebsch_gordan(*[S(i)  for i in args]))
            self._cache[args] = res
            
        return res

#   coeff(j1,j2,J,m1,m2,M)

coeff = ClebschGordan()

In [9]:
# # Example

# coeff(1/2,1/2,0,1/2,-1/2,0)

### print in scientific notation

In [10]:
def format_e(n, pre = 2, exp_dig = 2,ret_value = False):
    n = complex(n)
    n_str= np.format_float_scientific(n.real, precision = pre, exp_digits=exp_dig)+' + i '+np.format_float_scientific(n.imag, precision = pre, exp_digits=exp_dig)
    if not ret_value:
        print(n_str)
    else:
        return n_str

In [11]:
# # Example

# format_e(12.34567+2j)

### round of numbers in a list

In [12]:
def round_n(list1,n=3):
    
    def round_(l1,n=3):
        lround = []

        for elem in l1:
            if type(elem) != str :
                elem_imag = round(elem.imag,n)
                
                if elem_imag == 0:
                    
                    lround.append(float(round(elem.real,n)))
                else:
                    lround.append(round(elem.real,n)+ elem_imag*1j)

            else:
                lround.append(elem)
        return lround
    if type(list1)==list:
        list1 = np.matrix(list1)
    
    row, col = list1.shape
    
    rlist = []
    
    l1 = np.matrix.tolist(list1)
    
    for i in range(row):
            rlist.append(round_(l1[i],n))
    return np.matrix(rlist)
     

In [13]:
# # Example

# def row():
#     return [100*random() + 100*random()*1j for i in range(2)]

# m1 =  np.matrix([row(),row()])

# print('m1:\n',m1,'\n---------\nround_n(m1):\n',round_n(m1,2))


### remove the exponential part: $ a \times 10^b \rightarrow [a,b]$

In [14]:
def expo_power(n):
    a       = '%E' % n
    return a.split('E')

In [15]:
# # Example

# expo_power(12.3456)

### equate a and b within precision $\delta = 10^{-12}$

In [16]:
def equal(a, b,delta=10 ** -12): 
    return np.isclose(float(a),float(b),atol = delta, rtol = 0.0)

In [17]:
# # Example

# a = 1.9 *10 ** -12
# b = 2.1 * 10 ** -12

# print(equal(a,b))

# del a,b

### $\hat{a} = 2 a + 1$

In [18]:
def h(a):
    return 2*a + 1

### $\delta_{a,b} $


In [19]:
def kron(a,b): 
    return (1 if a==b else 0)

### Melement = $\langle l' m_l'| S_{a =1,2,3}|l m_l \rangle$.

note that:
$$ S_x = \frac{T_{-1}^1-T_{+1}^1}{\sqrt{2}}$$
$$ S_y = -\frac{T_{-1}^1+T_{+1}^1}{\sqrt{2}i}$$
$$ S_z = T_{0}^1$$
and
$$\langle l' m_l'| S_{a =1,2,3}|l m_l \rangle = \langle l m_l, 1 (\pm,0)| l' m_l' \rangle \langle l' || T^1 ||l \rangle$$

In [20]:
def Melement(lp, mlp, a, l, ml):
    def e(L,M,p):
        return  sqrt( L*(L+1) - M*(M+p)  )

     
    if a==1 :
        ans= 1/2 * ( e(l,ml,1)* kron(mlp,ml+1) 
            + e(l,ml,-1) * kron(mlp,ml-1)  )
    elif a==2 :
        ans= (1/2j )* ( e(l,ml,1)* kron(mlp,ml+1) 
            - e(l,ml,-1) * kron(mlp,ml-1)  )
    elif a==3 :
        ans= ml *kron(ml,mlp)
        
    return complex(ans * kron(l,lp))


In [21]:
# # Example

# Melement(1, 0, 1, 1, 1)

### spin states

In [22]:
# spin 1/2 states
spin_half = [-1/2,1/2]

# spin 1 states
spin_one = [ -1, 0 , 1]

### Spherical harmonics

In [23]:
def ylm(l,m,theta,phi):
    ans     = 0

    if abs(m)<=l and l>=0 :
        ans = sph_harm(m,l,phi,theta)

    return ans 

In [24]:
# ylm(0,0,0,0)/(1 / (2 * sqrt(pi)))

### angular grid

In [25]:
# angular grid
xgrid, wx = np.polynomial.legendre.leggauss(n_angle)

### Momentum grid

In [26]:
def mom_grid(  # pylint: disable=C0103
    n1: int = 10, n2 : int= 20, p1: float = 1.0, p2: float = 6.0, p3: float = 20.0
) -> Tuple[np.ndarray, np.ndarray]:
    r"""Allocates transformed Gauss-Legendre mesh for numerical integration.

    The total interval spans from p0 = 0.0 to p3. The points are distributed as follows:

    * n1 / 2 points in (p0, p1)
    * n1 / 2 points in (p1, p2)
    * n2  points in (p2, p3)

    The (p0, p2) interval follows the hyperbolic transformation
    $$
    x\_1 \\to
    \\frac{1 + x\_1}
    {\\frac{1}{p\_1} - x\_1\\left(\\frac{1}{p\_1} -\\frac{1}{p\_2} \\right)}
    $$

    and the (p1, p2) interval follows the linear transformation
    $$ x\_2 \\to \\frac{p\_3 + p\_2}{2} + x\_2 \\frac{p\_3 - p\_2}{2}$$


    **Arguments**
        n1: int
            Number of meshpoints for first interval (hyperbolic transformation).

        n2: int
            Number of meshpoints for second interval (linear transformation).

        p1: float = 1.0
            Middle point (defined by number of points) of first interval.

        p2: float = 6.0
            End point of first interval.

        p3: float = 20.0
            End point of second interval.

    **Returns**
        p, wp: Tuple[np.ndarray, np.ndarray]
            The meshpoints and the integration weights.

    .. note:: This routine follows Andreas Nogga's implementation of the TRNS routine.
    """
    x = []
    wx = []

    xtmp, wtmp = leggauss(n1)
    for xi, wi in zip(xtmp, wtmp):
        xxi = 1 / p1 - (1 / p1 - 2 / p2) * xi
        x.append((1 + xi) / xxi)
        wx.append((2 / p1 - 2 / p2) * wi / xxi ** 2)

    xtmp, wtmp = leggauss(n2)
    p23 = (p3 - p2) / 2
    P23 = (p3 + p2) / 2
    for xi, wi in zip(xtmp, wtmp):
        x.append(P23 + p23 * xi)
        wx.append(p23 * wi)

    return np.array(x), np.array(wx)

# momentum grid
pgrid,  wp = mom_grid(n1, n2,p1,p2,p3)
Np = len(pgrid)
pgrid = np.append(pgrid,q0)

In [27]:
# plt.hist(pgrid, bins=40)
# plt.xlabel('Momentum (MeV)')
# plt.ylabel('frequency')
# plt.grid()
# plt.savefig('mom_mesh.pdf')
# plt.show()

# ! rm mom_mesh.pdf

### plotting graphs

In [28]:
def display(x,y_all,label_xy=['x','y'],legend_y=['' for i in range(10)],vert_line=[10,False],save=['name',False]):

    style=['-r','-b',':g','-.k','.m','--y','--navy','--grey','--orange','--deeppink']
    

    for i in range(len(y_all)):

        plt.plot(x,y_all[i],style[i],label=legend_y[i])

    plt.ylabel(label_xy[1])
    plt.xlabel(label_xy[0])
    if legend_y[0]!='':
        plt.legend()
        
    if vert_line[1]:
        plt.axvline(x = vert_line[0], color = 'k')
    # plt.ylim(y_lim[0],y_lim[1])
    # plt.xlim(x_lim[0],x_lim[1])
    
    if save[1]:
        plt.savefig(save[0], format="pdf", bbox_inches="tight")
    plt.show()

In [29]:
# Example

# x = [i for i in range(50)]
# y = [[i**2 for i in range(50)],[i**2.5 for i in range(50)]]

# display(x,y,legend_y = ['y1','y2'],vert_line=[20,True])
# del x,y

### print first 10 rows of a dictionary

In [30]:
def print_(dict_,pre=3,dataframe=False):
    
    if dataframe:
        keys = list(dict_.keys())
    else:
        keys = list(islice(dict_.keys(),5))
    
    
    if len(keys[0])==6:
        col_name = ['s1','l1','j1','s2','l2','j2','value']
    elif len(keys[0])==8:
        col_name = ['s1','l1','j1','s2','l2','j2','p_grid 1','p_grid 2','value']
    else:
        col_name = []
        for i in range(len(keys[0])):
            col_name.append('col_'+str(i))
        col_name.append('value')
    
    d1 = {col:[] for col in col_name}
    for key in keys:
        i = 0
        for col in col_name:
            if col != 'value':
                d1[col].append( key[i])
                i+=1
            else:
                d1[col].append(format_e(dict_[key],pre, ret_value=True))
        
    
    print(pd.DataFrame(d1,columns=col_name).head())
    
    if dataframe:
        return pd.DataFrame(d1,columns=col_name)

In [31]:
# # Example
# dict_1 ={}
# for i in range(20):
#     dict_1[i,2*i,3*i,4*i] = random() 

# print_(dict_1)

### Print in $^{2S+1}L_J$ format

In [32]:
def phase_name(s,l,j):

    if l==0:
        orbit = 'S'
    elif l==1:
        orbit = 'P'
    elif l==2:
        orbit = 'D'
    elif l==3:
        orbit = 'F'
    elif l==4:
        orbit = 'G'
    else:
        orbit = 'X'

    return '{:}{:}{:}'.format(2*s+1,orbit,j)

## possible partial waves for a given j

In [33]:
# generate the non-zero (s,l,j) phases for j={0,1,2,3}

phase_dict={}

for j in range(4):
    phase_dict[j]=[]
    for s in range(2):
        for l in range(4):
            if abs(s-l)<= j <= s+l:
                phase_dict[j].append([s,l,j])


In [34]:
# # Example

# for phase in phase_dict[1]:
#     print(phase_name(*phase), " :  ",phase)

### Read and write pot_even.csv

In [35]:
class Vtensor_csv:
    
    d0 = {}
    d0['s1'] = []
    d0['l1'] = []
    d0['j1'] = []
    d0['s2'] = []
    d0['l2'] = []
    d0['j2'] = []
    d0['p_grid 1'] = []
    d0['p_grid 2'] = []
    d0['pot_even'] = []
    
    def __init__(self):
        
        if exists('pot_even.csv'):

            df_init = pd.read_csv('pot_even.csv',index_col=None)
            d1 = df_init.to_dict('list')
            self.Vtensor_ref = {}
            for i in range(len(d1['s1'])):
                self.Vtensor_ref[d1['s1'][i],d1['l1'][i],d1['j1'][i],d1['s2'][i],d1['l2'][i],d1['j2'][i],d1['p_grid 1'][i],d1['p_grid 2'][i]] = d1['pot_even'][i]
                

        else:
            self.Vtensor_ref = {}
    
    def read(self):
        return self.Vtensor_ref
    
    
    def write(self,Vtensor_dict):
        
        d1 = self.d0
    
    
        for key in Vtensor_dict:
            d1['s1'].append(key[0])
            d1['l1'].append(key[1])
            d1['j1'].append(key[2])
            d1['s2'].append(key[3])
            d1['l2'].append(key[4])
            d1['j2'].append(key[5])
            d1['p_grid 1'].append(key[6])
            d1['p_grid 2'].append(key[7])
            d1['pot_even'].append(Vtensor_dict[key])

        df1 = pd.DataFrame(d1)
        df1.to_csv('pot_even.csv',index=False)
        
    def delete(self):
        
        d1 = self.d0
    
        df1 = pd.DataFrame(d1)
        df1.to_csv('pot_even.csv',index=False)
        
    
Vfile = Vtensor_csv()

Vtensor_ref = Vfile.read()
    
    

In [36]:
# # Vtensor_ref details

# df1 =print_(Vtensor_ref,dataframe=True)

# print('\n\nj values : ',df1['j1'].unique())
# print('number of pi, pj values : ',len(df1['p_grid 1'].unique()))

# print(len(df1))

## To make the output auto-scroll to the lastest output
### Useful while calculating $C_0$

In [37]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

## $\Lambda$-values for plotting

In [38]:
lambda_values = [round(i,1) for i in np.arange(2.0,52.1,.1)]

In [39]:
# lambda_values